In [42]:
import requests
import csv
import time
import pandas as pd
from osm_script import osm_extractor_groups, fetch_osm_region
from reverse_geocode import geocode_lat_lon
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re 

In [3]:
#Several APIs available 

main_overpass_api = "https://overpass-api.de/api/interpreter"
lz4_overpass_api='https://lz4.overpass-api.de/api/interpreter'
osm_overpass_api='https://overpass.openstreetmap.ru/api/interpreter'

BUNDES_GROUPS = [
    ["Baden-Württemberg"],
    ["Bayern"],
    ["Berlin", "Bremen"],
    ["Brandenburg"],
    ["Hamburg"],
    ["Hessen","Mecklenburg-Vorpommern","Rheinland-Pfalz"],
    ["Niedersachsen"],
    ["Nordrhein-Westfalen"],
    ["Saarland","Sachsen","Sachsen-Anhalt", "Schleswig-Holstein","Thüringen"]
]

In [ ]:
df_raw=osm_extractor_groups(BUNDES_GROUPS) #for first time getting the data



In [2]:

#for later import the latest file 
df_raw = pd.read_csv("awo_20250926-135243_osm_raw.csv")
df_raw.shape

(4604, 14)

In [23]:
#it seems like empty cells contain empty string, therefore it needs to be replaced with NA values 
#df_raw = df_raw.replace(r'^\s*$', pd.NA, regex=True)
df_raw.sample(10)

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
1639,96453980,Mecklenburg-Vorpommern,way,Die Kleinen Schulzen,Schulzenweg,10,19061.0,Schwerin,53.609547,11.386542,NaN,NaN,https://www.awo-schwerin.de/fachbereiche/kinde...,kindergarten
3488,1225190712,Nordrhein-Westfalen,way,AWO KiTa im Monfortsquartier,Im Winkel,,41238,Mönchengladbach,51.181071,6.450341,NaN,NaN,NaN,kindergarten
2040,82265814,Niedersachsen,way,Wohnanlage am Mühlenhof,Im Kühlen Grunde,5,26180,Rastede,53.247303,8.211957,+49 4402 929190,NaN,https://www.wohnanlage-rastede.de,social_facility
1502,338182815,Hessen,way,AWOPLAN,Hersfelder Straße,39,36320.0,Kirtorf,50.760342,9.126503,+49 6635 918290,NaN,NaN,NaN
626,82923094,Bayern,way,AWO-Kinderhaus Meilenstein,Goethestraße,50,84032.0,Landshut,48.557548,12.128506,NaN,NaN,NaN,kindergarten
2775,26659361,Nordrhein-Westfalen,way,AWO Seniorenzentrum Witten,Egge,"73,75,77",58453.0,Witten,51.435158,7.353515,+49 2302 910450,sz-witten@awo-ww.de,https://sz-witten.awo-ww.de/,social_facility
2388,1154056361,Nordrhein-Westfalen,node,AWO Haus Bittermark,Sichelstraße,,44229,Dortmund,51.445900,7.467408,NaN,NaN,NaN,NaN
3509,1291691665,Nordrhein-Westfalen,way,Kindertagesstätte Sterntaler,Epprather Straße,,50181,Bedburg,51.007686,6.561118,+49 2272 3245,sterntaler@awo-bm-eu.de,https://www.awo-bm-eu.de/kinder/kitas/27-Stern...,kindergarten
329,1417095985,Bayern,node,Kita Schoppershof,Schoppershofstraße,82,90489.0,Nürnberg,49.465852,11.097427,+49 911 80199670,kita-schoppershof@awo-nbg.de,https://www.awo-nuernberg.de/,kindergarten
2972,166035346,Nordrhein-Westfalen,way,AWO Unterbezirk Ennepe-Ruhr e.V.,Neustraße,10,58285.0,Gevelsberg,51.320399,7.337124,NaN,NaN,NaN,NaN


In [3]:
df_raw.region.value_counts()

region
Nordrhein-Westfalen       1236
Bayern                     679
Niedersachsen              513
Thüringen                  300
Baden-Württemberg          299
Sachsen                    281
Hessen                     237
Schleswig-Holstein         190
Brandenburg                163
Mecklenburg-Vorpommern     156
Sachsen-Anhalt             142
Saarland                   121
Rheinland-Pfalz             94
Berlin                      84
Bremen                      69
Hamburg                     40
Name: count, dtype: int64

In [4]:
df_raw.isnull().sum() 

osm_id            0
region            0
type              0
name            413
street         1705
housenumber    1740
postcode       1825
city           1842
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

Reverse geocoding to fill address for rows having only lat and lon 


In [5]:
mask_empty_address = df_raw['city'].isna()
dict_empty_address = df_raw.loc[mask_empty_address].apply(lambda row:geocode_lat_lon(row['lat'], row['lon']), axis=1)


RateLimiter caught an error, retrying (0/2 tries). Called with (*((53.8516699, 10.6598188),), **{'language': 'de', 'addressdetails': True}).
Traceback (most recent call last):
  File "/home/tm/anaconda3/envs/immo/lib/python3.13/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "/home/tm/anaconda3/envs/immo/lib/python3.13/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/home/tm/anaconda3/envs/immo/lib/python3.13/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
TimeoutError: timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/tm/anaconda3/envs/immo/lib/python3.13/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response

In [6]:
add_addresses = pd.DataFrame(list(dict_empty_address), index=dict_empty_address.index)
df_raw.loc[mask_empty_address, add_addresses.columns] = add_addresses

/tmp/ipykernel_4426/3849740406.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['76275' '79331' '73525' ... '37327' '99427' '36448']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_raw.loc[mask_empty_address, add_addresses.columns] = add_addresses


In [7]:
df_raw.isna().sum()

osm_id            0
region            0
type              0
name            413
street           16
housenumber      47
postcode         18
city              0
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

In [8]:
df_raw[(df_raw['name'].isna()) & (df_raw['email'].notna())] # check which faclities have no name but have email and fill those values with email domain

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity
1276,1104913364,Brandenburg,way,NaN,Pietschkerstraße,44,14480.0,Potsdam,52.378131,13.125325,+49 331 887 49580,pfiffikus@awo-potsdam.de,https://awo-potsdam.de/de/standort/kita-pfiffi...,kindergarten
1994,12327386579,Niedersachsen,node,NaN,Dorfstraße,58a,21365.0,Adendorf,53.287278,10.445362,+49 4131 898 00 78,info@awosozial.de,NaN,social_facility
2304,1341369737,Niedersachsen,way,NaN,Im Tale,2-4,NaN,Celle,52.654001,10.070353,NaN,kita.imtale.celle@awo-juki.de,https://www.awo-juki.de/index.php?id=930&L=it%...,kindergarten
3847,273517107,Sachsen,way,NaN,Gerichtsstraße,2,2779.0,Großschönau,50.896862,14.665772,+49 35841 2430,kinderland@awo-oberlausitz.de,https://awo-oberlausitz.de/kindertagesstaetten...,childcare
4416,124874406,Thüringen,way,NaN,Mörlaer Straße,8b,07407,Rudolstadt,50.719988,11.322286,+49 3672 422 552,feste-burg@awo-rudolstadt.de,http://www.awo-rudolstadt.de,NaN
4444,181239665,Thüringen,way,NaN,Löwentorstraße,33,99752.0,Bleicherode,51.442816,10.569425,+49 36338 42486,leiterin@kita-bleicherode.de,NaN,NaN


In [9]:
df_raw.loc[df_raw['name'].isna(), 'name'] = df_raw['email'].str.split('@').str[1]

In [10]:
df_raw.isna().sum() # postcode is missing, same function for reverse geocoding needs t obe applied with rows having NaN for postcode , street and housenumber

osm_id            0
region            0
type              0
name            407
street           16
housenumber      47
postcode         18
city              0
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

In [11]:
postcode_mask = df_raw['postcode'].isna()
postcodes = df_raw.loc[postcode_mask].apply(lambda row: geocode_lat_lon(row['lat'], row['lon']), axis=1)
add_postcodes = pd.DataFrame(list(postcodes), index=postcodes.index)


In [12]:
df_raw.loc[postcode_mask, add_postcodes.columns] = add_postcodes

In [13]:
street_mask = df_raw['street'].isna()
streets = df_raw.loc[street_mask].apply(lambda row: geocode_lat_lon(row['lat'], row['lon']), axis=1)
add_streets = pd.DataFrame(list(streets), index=streets.index)

In [14]:
df_raw.loc[street_mask, add_streets.columns] = add_streets

In [15]:
housenumber_mask = df_raw['housenumber'].isna()
hnumbers =df_raw.loc[housenumber_mask].apply(lambda row: geocode_lat_lon(row['lat'], row['lon']), axis=1)
add_hnumbers = pd.DataFrame(list(hnumbers), index=hnumbers.index)
df_raw.loc[housenumber_mask, add_hnumbers.columns] = add_hnumbers

In [16]:
df_raw.isna().sum() 

osm_id            0
region            0
type              0
name            407
street            0
housenumber       0
postcode          0
city              0
lat               0
lon               0
phone          3138
email          3827
website        2846
amenity        1308
dtype: int64

In [25]:
df_raw[df_raw['name'].isna()]['amenity'].value_counts()

amenity
recycling                 128
parking                    64
kindergarten               25
social_facility            11
childcare                   7
bicycle_parking             7
social_centre               3
atm                         2
charging_station            2
nursing_home                2
fountain                    1
shelter                     1
bicycle_repair_station      1
community_centre            1
waste_basket                1
events_venue                1
drinking_water              1
waste_disposal              1
vending_machine             1
cafe                        1
give_box                    1
public_bookcase             1
parking_entrance            1
Name: count, dtype: int64

In [25]:
current_db = pd.read_excel("2025_09_16_Einrichtunsdatenbank_Export_descriptions_final.xlsx" , sheet_name = 'Facilities')
current_db.sample(6)

,verband_id,name,rechtsform,adresse_strasse,adresse_zusatz,adresse_plz,adresse_ort,adresse_telefon,adresse_email,postfach_adresse,...,bagfw_kategorie,anzahl_vollzeit,anzahl_teilzeit,anzahl_ehrenamt,anzahl_plaetze,anzahl_fsj,anzahl_bfd,target_group,tags,carrier_id
4094,3073,Mittagsbetreuung an der Grundschule Landau,NaN,Maria-Ward-Platz 2,NaN,94405,Landau,09951 - 6555,NaN,NaN,...,2.2.3,0,1,0,25,0,0,NaN,"Erholungshilfe,Kinder,Jugendliche",1.050000e+10
7704,2336,Mutter-Kind-Kuren,NaN,Chérisystr. 15,NaN,78467,Konstanz,07531 / 958963,anonymisiert@awo-konstanz.de,NaN,...,1.3.4,0,1,0,0,0,0,NaN,"Beratungsstellen,Mutter-Kind,Vater-Kind,Kur,Ku...",9.200600e+09
7132,11743,Tagesgruppe Neumühl Bauspielplatz Neumühl,NaN,Alexstraße 8,NaN,47167,Duisburg,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,NaN,"Kinder,Tagespflege,Kindergarten,Kindergärten,K...",6.300102e+09
8299,3478,AWO Integrations- und Service gGmbH AWO Integr...,NaN,Friedrich-Schiller-Straße 2 E,NaN,3127,Guben,03561 685170-0,anonymisiert@awo-waescherei.de,NaN,...,5.2.18,0,0,0,0,0,0,NaN,"Integrationsbetriebe,Behinderung",1.302000e+10
4174,3003,Seniorenzentrum Mömlingen AWO Care gGmbH,NaN,Danziger Straße 8,NaN,63853,Mömlingen,06022/7093210,anonymisiert@awo-unterfranken.de,NaN,...,4.1.4,0,0,0,68,0,0,NaN,"Altenpflege,Kurzzeitpflege,Seniorenzentrum,Sen...",1.040000e+10
4091,2509,Offene Ganztagsschule an der Ludwig-Thoma-Haup...,NaN,Dr. Engert-Straße 5,NaN,85221,Dachau,NaN,anonymisiert@awo-dachau.de,NaN,...,10.4.2.3,0,0,0,0,0,0,NaN,"Selbsthilfegruppe,Hausaufgaben,Schülergruppen",1.010040e+10


In [27]:
current_db.columns

Index(['verband_id', 'name', 'rechtsform', 'adresse_strasse', 'adresse_zusatz',
       'adresse_plz', 'adresse_ort', 'adresse_telefon', 'adresse_email',
       'postfach_adresse', 'postfach_plz', 'postfach_ort',
       'ansprechperson_anrede', 'ansprechperson_vorname',
       'ansprechperson_nachname', 'ansprechperson_funktion',
       'ansprechperson_email', 'ansprechperson_telefon', 'bagfw_kategorie',
       'anzahl_vollzeit', 'anzahl_teilzeit', 'anzahl_ehrenamt',
       'anzahl_plaetze', 'anzahl_fsj', 'anzahl_bfd', 'target_group', 'tags',
       'carrier_id'],
      dtype='object')

In [29]:
df_raw.columns

Index(['osm_id', 'region', 'type', 'name', 'street', 'housenumber', 'postcode',
       'city', 'lat', 'lon', 'phone', 'email', 'website', 'amenity'],
      dtype='object')

It could be hard to compare since names might differ : Kita or Kindertagesstätte or AWO-Kindertagesstätte, then names are under " " while in osm file are not or somewhere is given AWO and somewhere Arberiterwohlfahrt, then some names have "-" in between while others just space (example "Rhein-Au" vs. "Rhein Au"). This might be because db file contains official names while osm is filled manually by users.  

One option is to normalize the strings in name and compare, or since lot of names are missing, create address field in osm results and compare with address from source facility db . Also names need to be cleaned since some findings are not AWO facilities 

In [17]:
df_raw.to_csv('awo_osm_data.csv', index=False, encoding='utf-8')

In [38]:
NAMES_MAPPING ={
    "arbeiterwohlfahrt": "awo",
    "kindertagesstätte" : "kita",
    "eingetragener verein" : "e.v.",
    "evangelisch": "ev.",
    "kreisverband":"kv",
    "ortsverein": "ov",
    "altersheim": "altenpflegeheim",
}

ADDRESS_MAPPING ={
    "str.": "straße",
    "pl." : "platz",

} 

In [39]:
from rapidfuzz import fuzz, process

def normalize_text(text:str, rules:dict) ->str:
    if pd.isna(text):
        return " "
    text = text.lower()
    text = re.sub(r"[^a-z0-9äüöß]", " ", text)
    for k, v in rules.items():
        text=text.replace(k, v)
    return re.sub(r"\s+", " ", text).strip()

def normalize_name(text:str) ->str:
    return normalize_text(text, NAMES_MAPPING)

def normalize_address(addr:str) ->str:
    return normalize_text(text, ADDRESS_MAPPING)



In [44]:
def find_best_match(name, osm_names, threshold = 85):
    if not name:
        return None, 0
    match = process.extractOne(
        name,
        osm_names,
        scorer=fuzz.token_sort_ratio
    )
    if match and match[1] >= threshold:
        return match[0], match[1]
    return None, 0


def match_facilities(df_db, df_osm, threshold=85):
    #1.normalize names in both dfs
    df_db['name_norm'] = df_db['name'].apply(normalize_name)
    df_osm['name_norm'] = df_osm['name'].apply(normalize_name)

    osm_names = df_osm["name_norm"].dropna().unique().tolist() 

    # Match facilities DB → OSM
    df_db[["osm_match", "match_score"]] = df_db["name_norm"].apply(
        lambda x: pd.Series(find_best_match(x, osm_names, threshold)))

    df_db['found_in_osm']= df_db["osm_match"].notna()

    # Match OSM → facilities DB
    db_names = df_db['name_norm'].dropna().unique().tolist()

    df_osm[['db_match', 'match_score']] = df_osm["name_norm"].apply(
        lambda x:pd.Series(find_best_match(x, db_names, threshold)))

    df_osm['found_in_db'] = df_osm["db_match"].notna()

    return df_db, df_osm



In [45]:
current_db_matched, df_raw_matched = match_facilities(current_db, df_raw)

In [47]:
current_db['found_in_osm'].value_counts()

found_in_osm
False    8051
True     1977
Name: count, dtype: int64

In [48]:
df_raw['found_in_db'].value_counts()

found_in_db
False    2923
True     1681
Name: count, dtype: int64

In [49]:
df_raw.sample(10)

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity,name_norma,name_norm,db_match,match_score,found_in_db
1376,4405976595,Hessen,node,AWO Kindertagesstätte Bockenheim,Hersfelder Straße,14,60487.0,Frankfurt am Main,50.122683,8.635281,+49 69 95643591,awo.kita.bockenheim@awo-frankfurt.de,https://www.awo-frankfurt.de/kita-bockenheim,kindergarten,awo kita bockenheim,awo kita bockenheim,kita bockenheim,88.235294,True
1831,322902394,Niedersachsen,node,Volksbank BraWo,Wittenbergstraße,6,38124,Braunschweig,52.230490,10.535443,NaN,NaN,NaN,bank,volksbank brawo,volksbank brawo,NaN,0.000000,False
2550,5872348982,Nordrhein-Westfalen,node,die Tafeln,Beethovenstraße,33,53773.0,Hennef (Sieg),50.772962,7.279435,+49 2242 933 36 13,NaN,http://awo-ov-hennef.de/die-tafeln/,social_facility,die tafeln,die tafeln,NaN,0.000000,False
448,6070979557,Bayern,node,AWO Haus für Kinder,Hiroshimaplatz,,90461,Nürnberg,49.432643,11.098451,NaN,kita-nuerbanum@awo-nbg.de,NaN,kindergarten,awo haus für kinder,awo haus für kinder,NaN,0.000000,False
405,4240851900,Bayern,node,"AWO Kinderhaus ""Spatzennest""",Westring,6,85614.0,Kirchseeon,48.080083,11.854791,+49 8091 6613,NaN,https://awo-kv-ebe.de/kiju/kh/eglharting/index...,kindergarten,awo kinderhaus spatzennest,awo kinderhaus spatzennest,NaN,0.000000,False
221,323467814,Baden-Württemberg,way,AWO Schülerhort,Hieronymus-Nopp-Straße,3,76661.0,Philippsburg,49.233471,8.460427,NaN,NaN,NaN,social_facility,awo schülerhort,awo schülerhort,NaN,0.000000,False
3538,1374137355,Nordrhein-Westfalen,way,NaN,Lönsweg,2,49504,Lotte,52.276630,7.924071,NaN,NaN,NaN,kindergarten,,,NaN,0.000000,False
3872,476872250,Sachsen,way,AWO Seniorenzentrum,August-Bebel-Straße,10,4769.0,Mügeln,51.233190,13.037287,NaN,NaN,NaN,social_facility,awo seniorenzentrum,awo seniorenzentrum,awo seniorenzentrum,100.000000,True
1726,9482420,Mecklenburg-Vorpommern,relation,AWO Sportinternat und Mensa,Von-Flotow-Straße,22,19059.0,Schwerin,53.625861,11.388920,NaN,NaN,https://www.sportgymnasium-schwerin.de/?page_i...,NaN,awo sportinternat und mensa,awo sportinternat und mensa,NaN,0.000000,False
4566,899151355,Thüringen,way,"AWO Kinder- und Jugendhaus ""Sicor""",Königshofener Straße,21b,7607.0,Eisenberg,50.975076,11.898705,NaN,NaN,NaN,social_facility,awo kinder und jugendhaus sicor,awo kinder und jugendhaus sicor,NaN,0.000000,False


In [51]:
df_raw_matched[df_raw_matched['found_in_db']==False].sample(20)

,osm_id,region,type,name,street,housenumber,postcode,city,lat,lon,phone,email,website,amenity,name_norma,name_norm,db_match,match_score,found_in_db
2556,5975088184,Nordrhein-Westfalen,node,HaWoS Immobilien & Hausverwaltung,Spielplatzstraße,25,59555,Lippstadt,51.672570,8.344922,NaN,NaN,NaN,NaN,hawos immobilien hausverwaltung,hawos immobilien hausverwaltung,NaN,0.0,False
3516,1303138030,Nordrhein-Westfalen,way,KiTa Südhafen,Scheibenstraße,36,48153,Münster,51.944524,7.624434,NaN,NaN,https://www.awo-msl-re.de/awo-vor-ort/muenster...,kindergarten,kita südhafen,kita südhafen,NaN,0.0,False
2229,761352451,Niedersachsen,way,Arbeiterwohlfahrt Kreisverband Cloppenburg,Lange Straße,21,26676.0,Barßel,53.169376,7.744022,NaN,NaN,NaN,NaN,awo kv cloppenburg,awo kv cloppenburg,NaN,0.0,False
639,96936964,Bayern,way,NaN,Römerweg,50,86199.0,Augsburg,48.336294,10.869361,NaN,NaN,NaN,NaN,,,NaN,0.0,False
1720,1376413946,Mecklenburg-Vorpommern,way,Bi uns to Hus,Rosengarten,,17367,Eggesin,53.681861,14.077063,NaN,NaN,NaN,social_facility,bi uns to hus,bi uns to hus,NaN,0.0,False
2095,166979265,Niedersachsen,way,AWO Residenz Gehrden,Thiemorgen,1,30989.0,Gehrden,52.313447,9.610314,NaN,NaN,NaN,social_facility,awo residenz gehrden,awo residenz gehrden,NaN,0.0,False
3421,1019038989,Nordrhein-Westfalen,way,"Kindertagesstätte ""Lärchenhof""",Katharinenstraße,47,52353.0,Düren,50.845860,6.429775,NaN,NaN,https://awo-dn.de/kindertageseinrichtung-laerc...,kindergarten,kita lärchenhof,kita lärchenhof,NaN,0.0,False
4529,624056807,Thüringen,way,Glühwürmchen,Schwerborner Straße,30,99087.0,Erfurt,51.014337,11.039191,NaN,NaN,NaN,kindergarten,glühwürmchen,glühwürmchen,NaN,0.0,False
4531,656782131,Thüringen,way,Kindergarten Im Ziegenhainer Tal,In der Doberau,4,7749.0,Jena,50.920714,11.605980,NaN,NaN,NaN,kindergarten,kindergarten im ziegenhainer tal,kindergarten im ziegenhainer tal,NaN,0.0,False
4434,158858303,Thüringen,way,"Seniorenpark ""Voigt'sches Gut""",Karl-Marx-Straße,28,4617.0,Rositz,51.016273,12.371967,NaN,NaN,NaN,nursing_home,seniorenpark voigt sches gut,seniorenpark voigt sches gut,NaN,0.0,False


In [ ]:
#name_datetime = time.strftime("%Y%m%d-%H%M%S")
df_raw.to_csv(f"awo_{name_datetime}_osm_raw.csv", index=False, encoding='utf-8')